In [3]:
import pandas as pd
from datetime import datetime
import sqlite3
import hashlib
import ssl
from urllib.error import HTTPError
ssl._create_default_https_context = ssl._create_unverified_context

## L_Team_Player

In [61]:
# Drop L_Team_Player
import sqlite3
data_base=f'D:/AFL_GRP/DataBase/AFLDataVault.db'
table_name='L_Team_Player'
connection=sqlite3.connect(database=data_base)
cursor = connection.cursor()
# SQL commands to drop and recreate the table
drop_query = f"DROP TABLE IF EXISTS {table_name};"
# Execute the drop and create queries
cursor.execute(drop_query)
# Commit the changes
connection.commit()
connection.close()

In [57]:
data_base=f'D:/AFL_GRP/DataBase/AFLStats.db'
table_name='ScrapedPlayerStatistics'
connection=sqlite3.connect(database=data_base)

query_ps= f''' 
select Player_Firstname,Player_Surname,Team from  scrapedPlayerStatistics 
group by Player_Firstname,Player_Surname,Team

'''

df_out=pd.read_sql(query_ps,connection)
connection.close()
df_out

,Player_Firstname,Player_Surname,Team
0,Aaron,Black,GE
1,Aaron,Black,NM
2,Aaron,Black,WC
3,Aaron,Cadman,GW
4,Aaron,Cornelius,BL
...,...,...,...
3563,Zak,Jones,SK
3564,Zak,Jones,SW
3565,Zane,Duursma,NM
3566,Zane,Trew,WC


In [58]:
# Drop L_Team_Player
import sqlite3
data_base=f'D:/AFL_GRP/DataBase/AFLDataVault.db'
table_name='Source_L_Team_Player'
connection=sqlite3.connect(database=data_base)
cursor = connection.cursor()
# SQL commands to drop and recreate the table
drop_query = f"DROP TABLE IF EXISTS {table_name};"
# Execute the drop and create queries
cursor.execute(drop_query)
# Commit the changes
connection.commit()
connection.close()

In [59]:
data_base=f'D:/AFL_GRP/DataBase/AFLDataVault.db'
table_name='Source_L_Team_Player'
connection=sqlite3.connect(database=data_base)
df_out.to_sql(table_name, connection, index=False, if_exists='append')
connection.close()
df_out

,Player_Firstname,Player_Surname,Team
0,Aaron,Black,GE
1,Aaron,Black,NM
2,Aaron,Black,WC
3,Aaron,Cadman,GW
4,Aaron,Cornelius,BL
...,...,...,...
3563,Zak,Jones,SK
3564,Zak,Jones,SW
3565,Zane,Duursma,NM
3566,Zane,Trew,WC


In [62]:
import pandas as pd
import sqlite3
import hashlib

def hash_row(row):
    # Combine the values from the three columns, converting them to strings and handling NaN values
    combined = str(['Team'])+str(row['Firstname']) + str(row['Surname']) + str(row['DOB'])
    # Generate a hash using SHA-256
    return hashlib.sha256(combined.encode()).hexdigest()



data_base=f'D:/AFL_GRP/DataBase/AFLDataVault.db'
connection=sqlite3.connect(database=data_base)
query= f''' 
select  Ht.team_Hashkey
       ,hp.Player_Hashkey
       ,hp.Firstname
       ,hp.surname
       ,hp.dob
       
from   Source_L_Team_Player sltp
join   H_team ht
on     sltp.Team=ht.Team
join   h_Player hp
on     sltp.Player_Firstname=hp.firstname
and    sltp.Player_Surname=hp.surname

'''
df=pd.read_sql(query,connection)


connection.close()
df

df['LinkTeamPlayer_Hashkey'] = df.apply(hash_row, axis=1)
df['Load_Date']=datetime.now().date()
df['Record_Source']='ScrapedPlayerStatistics'
df=df[['LinkTeamPlayer_Hashkey','Team_Hashkey','Player_Hashkey','Load_Date','Record_Source']]
df

,LinkTeamPlayer_Hashkey,Team_Hashkey,Player_Hashkey,Load_Date,Record_Source
0,3a8e4464cc9f04f251c3f878bf1cd618b07f8e70aecba9...,e87601989d7d2997b3e9ebc455fb8fc4f6f65f46151e56...,384290e79019d004ec532e64c95e9216fd5f2281a16dd3...,2024-10-12,ScrapedPlayerStatistics
1,2645b56bbe4c1ab12bdb5647d710993a6f0fd2e42d80c7...,e87601989d7d2997b3e9ebc455fb8fc4f6f65f46151e56...,96f632fecbb0526e077263be4b899fa2ea6865b97a21ea...,2024-10-12,ScrapedPlayerStatistics
2,3a8e4464cc9f04f251c3f878bf1cd618b07f8e70aecba9...,b8ba3534c68e62c15aebf997e9baf09f0980f6501af376...,384290e79019d004ec532e64c95e9216fd5f2281a16dd3...,2024-10-12,ScrapedPlayerStatistics
3,2645b56bbe4c1ab12bdb5647d710993a6f0fd2e42d80c7...,b8ba3534c68e62c15aebf997e9baf09f0980f6501af376...,96f632fecbb0526e077263be4b899fa2ea6865b97a21ea...,2024-10-12,ScrapedPlayerStatistics
4,3a8e4464cc9f04f251c3f878bf1cd618b07f8e70aecba9...,fc233d527d220e5bea38fcff1ce0dcad5c371c81e0a800...,384290e79019d004ec532e64c95e9216fd5f2281a16dd3...,2024-10-12,ScrapedPlayerStatistics
...,...,...,...,...,...
3715,2314285f920c0e3a0114cd1da8ed20e46f3c071d8fba1d...,1ae55232c15b65f071015b34ba493dc4c0057cef485968...,ab828a19638145636db834422ec823d256b751a6e4133b...,2024-10-12,ScrapedPlayerStatistics
3716,2314285f920c0e3a0114cd1da8ed20e46f3c071d8fba1d...,072f005fc3f2a49f2e29225d116c10a3904c326a1b810a...,ab828a19638145636db834422ec823d256b751a6e4133b...,2024-10-12,ScrapedPlayerStatistics
3717,4ee20f72c9e0e3434bee3b5c8e1384287d16a2e564e55b...,b8ba3534c68e62c15aebf997e9baf09f0980f6501af376...,b6ed05383a8a0b9e08af63da919098c5c6448dcf5134e1...,2024-10-12,ScrapedPlayerStatistics
3718,c020b1c9ab8c9a2a56a8a607158fd1307c53987217b70e...,fc233d527d220e5bea38fcff1ce0dcad5c371c81e0a800...,a197230cc7666604a144510d3f757152c6092fd9bd1100...,2024-10-12,ScrapedPlayerStatistics


Load data into L_Team_Player

In [63]:
data_base=f'D:/AFL_GRP/DataBase/AFLDataVault.db'
table_name='L_Team_Player'
connection=sqlite3.connect(database=data_base)
df.to_sql(table_name, connection, index=False, if_exists='append')
connection.close()

Test L_Team_Player

There are Players with duplicate Firstname, Surname combinations need to be cleaned out.

In [65]:
data_base=f'D:/AFL_GRP/DataBase/AFLDataVault.db'
connection=sqlite3.connect(database=data_base)
query=f'''select LinkTeamPlayer_hashkey,team_hashkey,player_hashkey ,count(1)
from l_Team_player 
group by LinkTeamPlayer_hashkey,team_hashkey,player_hashkey
having count(1)>1  '''
df=pd.read_sql(query,connection)
connection.close()
df

,LinkTeamPlayer_Hashkey,Team_Hashkey,Player_Hashkey,count(1)


In [85]:
data_base=f'D:/AFL_GRP/DataBase/AFLDataVault.db'
connection=sqlite3.connect(database=data_base)
query= f''' 
select   ltp.LinkTeamPlayer_Hashkey
        ,ht.team
        ,hp.Firstname
        ,hp.Surname
        ,hp.dob
from    L_Team_Player ltp
join    H_Team ht
on      ltp.Team_Hashkey=ht.Team_Hashkey
join    H_Player hp
on      ltp.player_hashkey=hp.player_hashkey

where   Firstname='Bill'
and     Surname='Anderson'
--and     DOB='1-Oct-1961'
--and     Team in ('GC')
'''
# delete_query=f'''
# delete from L_Team_Player 
# where LinkTeamPlayer_Hashkey in (
#         select   ltp.LinkTeamPlayer_Hashkey
#         from    L_Team_Player ltp
#         join    H_Team ht
#         on      ltp.Team_Hashkey=ht.Team_Hashkey
#         join    H_Player hp
#         on      ltp.player_hashkey=hp.player_hashkey

#         where   Firstname='Gary'
#         and     Surname='Ablett'
#         and     DOB='1-Oct-1961'
#         and     Team in ('GC')
#         )
# '''
df=pd.read_sql(query,connection)
# cursor = connection.cursor()
# cursor.execute(delete_query)
# connection.commit()
connection.close()
df

,LinkTeamPlayer_Hashkey,Team,Firstname,Surname,DOB


The following players may cause issue.  Need to sort out how to get rid of L_Team_Player that are not reality

In [86]:
data_base=f'D:/AFL_GRP/DataBase/AFLDataVault.db'
table_name='H_Player'
connection=sqlite3.connect(database=data_base)
query_ps= f''' 
select firstname,surname,count(1) from H_Player
where 1=1
group by firstname,surname
having count(1) >1
order by Surname
'''
df_out=pd.read_sql(query_ps,connection)
connection.close()
df_out

,Firstname,Surname,count(1)
0,Gary,Ablett,2
1,Bill,Ahern,2
2,Frank,Aked,2
3,Bill,Anderson,2
4,Bruce,Anderson,2
...,...,...,...
451,Brian,Wilson,2
452,Stan,Wilson,2
453,Peter,Wood,2
454,Charlie,Young,2
